In [2]:
import tensorflow as tf

In [11]:
x = tf.random.normal([1,4,4,1], 0, 1, tf.float32)


print(x.shape)

y = tf.keras.layers.MaxPooling2D((2, 2))(x)
y2 = tf.keras.layers.MaxPooling2D(2, 2)(x)

print(y)
print(y2)

(1, 4, 4, 1)
tf.Tensor(
[[[[1.8174407 ]
   [0.98337185]]

  [[0.78887016]
   [0.79055876]]]], shape=(1, 2, 2, 1), dtype=float32)
tf.Tensor(
[[[[1.8174407 ]
   [0.98337185]]

  [[0.78887016]
   [0.79055876]]]], shape=(1, 2, 2, 1), dtype=float32)


In [14]:
import tensorflow as tf
import numpy as np
from layers import ReshapeTensor, ExpandDimensions

inputs = tf.random.normal([10,8,8,2048])

depth = 256
atrous_rates = (1,2,3)
kernel_size = 3
stride = 1
rate = 1

depth_padding = 'same'

inputs_size = tf.shape(inputs)[1:3]

conv_1x1 = tf.keras.layers.Conv2D(depth, 1, strides=1, padding=depth_padding, dilation_rate=1, use_bias = False)(inputs)
conv_3x3_1 = tf.keras.layers.Conv2D(depth, 3, strides=1, padding=depth_padding, dilation_rate=atrous_rates[0], use_bias = False)(inputs)
conv_3x3_2 = tf.keras.layers.Conv2D(depth, 3, strides=1, padding=depth_padding, dilation_rate=atrous_rates[1], use_bias = False)(inputs)
conv_3x3_3 = tf.keras.layers.Conv2D(depth, 3, strides=1, padding=depth_padding, dilation_rate=atrous_rates[2], use_bias = False)(inputs)

print(np.shape(inputs))
# global average pooling


image_level_features = tf.keras.layers.GlobalAveragePooling2D(keepdims=True)(inputs)
print(np.shape(image_level_features))

image_level_features = tf.keras.layers.Conv2D(depth, 1, strides=1)(image_level_features)
        
#bilinearly upsample features
image_level_features = ReshapeTensor(inputs_size)(image_level_features)

print(np.shape(image_level_features))

net = tf.keras.layers.Concatenate(axis=3)([conv_1x1, conv_3x3_1, conv_3x3_2, conv_3x3_3, image_level_features])        
print(net.shape)
net = tf.keras.layers.Conv2D(depth, 1, strides=1)(net)
print(net.shape)
 

(10, 8, 8, 2048)
(10, 1, 1, 2048)
(10, 8, 8, 256)
(10, 8, 8, 1280)
(10, 8, 8, 256)
